In [ ]:
%run _utils.ipynb

In [ ]:
# Built from the two cells below
gk_issues = load_gk_issues_for_analysis()
non_gk_issues = load_non_gk_issues_for_analysis()

In [ ]:
all_issues = load_all_issues()

In [ ]:
all_issues.info()

In [ ]:
temp = all_issues[~all_issues['is_pull_request']]

In [ ]:
temp2 = temp[temp['user_login'] == 'greenkeeper[bot]'].copy().reset_index(drop=True)
temp2.info()

In [ ]:

pattern = r'An in-range update of .* is breaking the build'

matched = temp2[temp2['title'].str.contains(pattern)]
not_matched = temp2[~temp2['title'].str.contains(pattern)]
not_matched = not_matched[~not_matched['title'].str.contains('Action required: Greenkeeper could not be activated')]
not_matched = not_matched[~not_matched['title'].str.contains('Version 10 of node.js has been released', regex=False)]
not_matched = not_matched[~not_matched['title'].str.contains('Payment required', regex=False)]
not_matched = not_matched[~not_matched['title'].str.contains('Invalid Greenkeeper configuration file', regex=False)]
not_matched = not_matched[~not_matched['title'].str.contains('Greenkeeper is no longer available for new installations', regex=False)]
not_matched = not_matched[~not_matched['title'].str.contains('Version 4 of node.js has been deprecated', regex=False)]
not_matched = not_matched[~not_matched['title'].str.contains('Implement Greenkeeper', regex=False)]
not_matched = not_matched[~not_matched['title'].str.contains('ci(pkg): enable Greenkeeper', regex=False)]
not_matched = not_matched[~not_matched['title'].str.contains('Drop support for node 4 / npm 2', regex=False)]

not_matched.info()

In [ ]:
not_matched['title'].value_counts()

In [ ]:
not_matched['repo_name'].value_counts().head(27)

In [ ]:
# issues = load_issues()
# all_issues = load_all_issues()
# counts = all_issues.groupby('repo_name')['id'].count()
# print(f"Count={len(all_issues)} (Project Count={len(counts)}) Median={counts.median()} Mean={counts.mean()}")
# for_plot = counts.to_frame(name='num_of_issues')
# for_plot['placeholder'] = ''
# for_plot['num_of_issues_log'] = np.log10(for_plot['num_of_issues'])
# fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,2))
# ax = sns.violinplot(
#     data=for_plot,
#     x='num_of_issues_log',
#     y='placeholder',
#     inner='quartile',
#     palette=['0.9'],
#     logscale=True
# )
# ax.set_ylabel('')
# ax.xaxis.set_major_formatter(log10_func_formatter)
# plt.show()

In [ ]:
# def filter_issues_for_analyses(all_issues, gk_issues):
#     # Make copies
#     gk_issues_for_analysis = gk_issues.copy()
#     non_gk_issues_for_analysis = all_issues.copy()
#     # Remove pull requests from all issues
#     non_gk_issues_for_analysis = non_gk_issues_for_analysis[
#         ~non_gk_issues_for_analysis['is_pull_request']
#     ]
#     # Filter out any issues that are GK in-range breakin gbuild issues
#     non_gk_issues_for_analysis = non_gk_issues_for_analysis[
#         ~non_gk_issues_for_analysis['id'].isin(gk_issues_for_analysis['id'])
#     ]
#     # Only analyze GK issues for projects we have all issues for
#     gk_issues_for_analysis = gk_issues_for_analysis[
#         gk_issues_for_analysis['repo_name'].isin(non_gk_issues_for_analysis['repo_name'])
#     ]
#     # Copy the html_url and body attributes over from all_issues to their respective rows on gk_issues
#     gk_issues_for_analysis = gk_issues_for_analysis.merge(
#         all_issues[['id', 'html_url', 'body']],
#         how='left',
#         on='id',
#     )
#     # Only select non-GKIRs that were created after the first GKIR was created in the project
#     first_gkir_created_at = (
#         gk_issues
#             .sort_values(['repo_name', 'created_at'], ascending=True)
#             .groupby('repo_name')['created_at']
#             .first()
#             .reset_index()
#     )
#     def foo(df, first_gkirs):
#         if len(df) == 0:
#             return pd.DataFrame()
#         try:
#             first_gkir = (
#                 first_gkirs[
#                     first_gkirs['repo_name'] == df.iloc[0]['repo_name']
#                 ]['created_at']
#                 .to_numpy()[0]
#             )
#         except Exception:
#             return pd.DataFrame()
#         return df[df['created_at'] >= first_gkir]
#     non_gk_issues_for_analysis = (
#         non_gk_issues_for_analysis
#             .groupby('repo_name')
#             .apply(lambda df: foo(df, first_gkir_created_at))
#             .reset_index(drop=True)
#     )
#     return non_gk_issues_for_analysis, gk_issues_for_analysis
    

# non_gk_issues, gk_issues = filter_issues_for_analyses(all_issues, issues)

# import re
# def extract_dependency_type(row):
#     try:
#         prior_cats = ['devDependency', 'dependency', 'optionalDependency', 'peerDependency']
#         if row['dependency_type'] in prior_cats:
#             return row['dependency_type']

#         body = row['body'].replace(' ', '')
#         pattern = '<thalign=left>\\nType\\n<\/td>\\n<td>\\n(.*)\\n<\/td>'
#         match = re.search(pattern, body)
#         if match is None:
#             return pd.NA
#         split_lines = match[0].splitlines()
#         if 'devDependency' in split_lines:
#             return 'devDependency'
#         elif 'dependency' in split_lines:
#             return 'dependency'
#         elif 'peerDependency' in split_lines:
#             return 'peerDependency'
#         else:
#             return pd.NA
#     except Exception:
#         return pd.NA

# gk_issues['dependency_type'] = gk_issues.apply(extract_dependency_type, axis=1)

# gk_issues.to_csv(f'{PROJECT_ROOT}/{CSV_FOLDER}/gk_issues_for_analysis.csv', index=False)
# non_gk_issues.to_csv(f'{PROJECT_ROOT}/{CSV_FOLDER}/non_gk_issues_for_analysis.csv', index=False)

## NOTE: These can be loaded using load_gk_issues_for_analysis() and load_non_gk_issues_for_analysis()

<h3>What data are we missing?</h3>

How many unique projects?

In [ ]:
gk_issues['repo_name'].nunique()

In [ ]:
gkir_repos = gk_issues[['repo_name']].drop_duplicates().reset_index(drop=True)
# stargazers_count['stargazers_count'] = ''
gkir_repos.to_csv(f'../csv/gkir_repos.csv', index=False)

In [ ]:
gkir_repos_info = pd.read_csv(f'../csv/gkir_repos_info.csv')
gkir_repos_info.describe()

How many unique issue ids?

In [ ]:
gk_issues['id'].nunique()

In [ ]:
non_gk_issues['id'].nunique()

In [ ]:
gk_issues['created_at'].hist()

What proportion are missing dependency name?

In [ ]:
missing_dependency_name = gk_issues[pd.isna(gk_issues['dependency_name'])]
len(missing_dependency_name) / len(gk_issues)

In [ ]:
missing_dependency_name['created_at'].hist()

What proportion are missing dependency name?

In [ ]:
missing_dependency_next_version = gk_issues[pd.isna(gk_issues['dependency_next_version'])]
len(missing_dependency_next_version) / len(gk_issues)

In [ ]:
missing_dependency_next_version['created_at'].hist()

What proportion are missing dependency type?

In [ ]:
missing_dependency_type = gk_issues[pd.isna(gk_issues['dependency_type'])]
len(missing_dependency_type) / len(gk_issues)

In [ ]:
missing_dependency_type['created_at'].hist()

In [ ]:
missing_dependency_type['url'].sample()